In [2]:
import os
import pandas as pd

First build the method to handle the ID.

In [2]:
ID_LENGTH = 15

def ID_build(ID_keyword, ID_sequence):
    FILL_LENGTH = ID_LENGTH - len(ID_keyword)
    ID = ID_keyword+str(ID_sequence).zfill(FILL_LENGTH)
    return ID

Testing the function. As shown below, the ID as expected lenth of 15.

In [3]:
Sample_id = ID_build('SLIDE',32)
print(Sample_id,str(len(Sample_id)))

SLIDE0000000032 15


Read all the datasets prepared

In [3]:
grid_data = pd.read_csv('../export_data/grid_data_base.csv')
slide_data = pd.read_csv('../export_data/slide_data_raw.csv')
specimen_data = pd.read_csv('../export_data/specimen_data.csv')
annotation_genus = pd.read_csv('../export_data/grid_annotation_genus.csv')
annotation_pseudo = pd.read_csv('../export_data/grid_annotation_pseudo.csv')
annotation_species = pd.read_csv('../export_data/grid_annotation_species.csv')

First handle the slide data. Two tables are expected to be created: Slide ID hub and slide_data. They are linked through the slide ID.

In [6]:
slide_data = pd.read_csv('slide_data_raw.csv')
slide_data.head()

,Core,Sub_core,Magnification,Height,Width,Path
0,F403,NaN,50X,4885,11714,\raw_images\F403_50X.tif
1,HK14DB1C,0_1,50X,9334,21409,\raw_images\HK14DB1C_0_1_50X.tif
2,HK14DB1C,104_105,50X,9326,21391,\raw_images\HK14DB1C_104_105_50X.tif
3,HK14DB1C,112_113(1),50X,9185,21471,\raw_images\HK14DB1C_112_113(1)_50X.tif
4,HK14DB1C,112_113(2),50X,9404,21290,\raw_images\HK14DB1C_112_113(2)_50X.tif


Considering the NaN value, we should replace the string fields with empty string and numerical fields with 0.

In [7]:
slide_data = slide_data.transform(lambda x: x.fillna('') if x.dtype == 'object' else x.fillna(0))
slide_data.head()

,Core,Sub_core,Magnification,Height,Width,Path
0,F403,,50X,4885,11714,\raw_images\F403_50X.tif
1,HK14DB1C,0_1,50X,9334,21409,\raw_images\HK14DB1C_0_1_50X.tif
2,HK14DB1C,104_105,50X,9326,21391,\raw_images\HK14DB1C_104_105_50X.tif
3,HK14DB1C,112_113(1),50X,9185,21471,\raw_images\HK14DB1C_112_113(1)_50X.tif
4,HK14DB1C,112_113(2),50X,9404,21290,\raw_images\HK14DB1C_112_113(2)_50X.tif


According to the case listed above, transform all table with the transformation to avoid the future problems.

In [8]:
grid_data = grid_data.transform(lambda x: x.fillna('') if x.dtype == 'object' else x.fillna(0))
specimen_data = specimen_data.transform(lambda x: x.fillna('') if x.dtype == 'object' else x.fillna(0))
annotation_genus = annotation_genus.transform(lambda x: x.fillna('') if x.dtype == 'object' else x.fillna(0))
annotation_pseudo = annotation_pseudo.transform(lambda x: x.fillna('') if x.dtype == 'object' else x.fillna(0))
annotation_species = annotation_species.transform(lambda x: x.fillna('') if x.dtype == 'object' else x.fillna(0))

As observed above, the Core, Sub_core and Magnification field will be used to create the ID.

In [9]:
slide_data['num_id'] = slide_data.groupby(['Path']).ngroup()

The numeric ID is increasing form 0. Therefore all the IDs could be generated by using the num_id.

In [10]:
slide_data['slide_id'] = slide_data.apply(lambda x: ID_build('SLIDE',x['num_id']), axis=1)

In [11]:
slide_data.head()

,Core,Sub_core,Magnification,Height,Width,Path,num_id,slide_id
0,F403,,50X,4885,11714,\raw_images\F403_50X.tif,0,SLIDE0000000000
1,HK14DB1C,0_1,50X,9334,21409,\raw_images\HK14DB1C_0_1_50X.tif,1,SLIDE0000000001
2,HK14DB1C,104_105,50X,9326,21391,\raw_images\HK14DB1C_104_105_50X.tif,2,SLIDE0000000002
3,HK14DB1C,112_113(1),50X,9185,21471,\raw_images\HK14DB1C_112_113(1)_50X.tif,3,SLIDE0000000003
4,HK14DB1C,112_113(2),50X,9404,21290,\raw_images\HK14DB1C_112_113(2)_50X.tif,4,SLIDE0000000004


Export the target data to csv form

In [4]:
slide_id_hub_cols = ['slide_id','Core','Sub_core', 'Magnification']
slide_data_cols = ['slide_id','Height','Width','Path']

In [13]:
slide_data.to_csv('slide_id_hub.csv', columns = slide_id_hub_cols, index=None)
slide_data.to_csv('slide_data.csv', columns = slide_data_cols, index=None)

Next handle grid related tables. First crated the id for the grids. Follows the sequence of the slide.

In [14]:
grid_data.head()

,Core,Sub_core,Grid number,Magnification,Height,Width,Grid Path
0,F403,,1,50X,910,895,\Grid_images\F403_50X\F403_50X_grid_1.tif
1,F403,,10,50X,895,905,\Grid_images\F403_50X\F403_50X_grid_10.tif
2,F403,,11,50X,905,904,\Grid_images\F403_50X\F403_50X_grid_11.tif
3,F403,,12,50X,900,910,\Grid_images\F403_50X\F403_50X_grid_12.tif
4,F403,,13,50X,890,905,\Grid_images\F403_50X\F403_50X_grid_13.tif


In [15]:
def build_id_from_column(unique_col_name, id_col_name, id_target, df):
    df['num_id'] = df.groupby([unique_col_name]).ngroup()
    df[id_col_name] = df.apply(lambda x: ID_build(id_target,x['num_id']), axis=1)

In [16]:
build_id_from_column('Grid Path', 'grid_id','GRID',grid_data)

In [17]:
grid_data.head()

,Core,Sub_core,Grid number,Magnification,Height,Width,Grid Path,num_id,grid_id
0,F403,,1,50X,910,895,\Grid_images\F403_50X\F403_50X_grid_1.tif,0,GRID00000000000
1,F403,,10,50X,895,905,\Grid_images\F403_50X\F403_50X_grid_10.tif,1,GRID00000000001
2,F403,,11,50X,905,904,\Grid_images\F403_50X\F403_50X_grid_11.tif,2,GRID00000000002
3,F403,,12,50X,900,910,\Grid_images\F403_50X\F403_50X_grid_12.tif,3,GRID00000000003
4,F403,,13,50X,890,905,\Grid_images\F403_50X\F403_50X_grid_13.tif,4,GRID00000000004


In [18]:
grid_data['status_code'] = 99

In [19]:
grid_data.head()

,Core,Sub_core,Grid number,Magnification,Height,Width,Grid Path,num_id,grid_id,status_code
0,F403,,1,50X,910,895,\Grid_images\F403_50X\F403_50X_grid_1.tif,0,GRID00000000000,99
1,F403,,10,50X,895,905,\Grid_images\F403_50X\F403_50X_grid_10.tif,1,GRID00000000001,99
2,F403,,11,50X,905,904,\Grid_images\F403_50X\F403_50X_grid_11.tif,2,GRID00000000002,99
3,F403,,12,50X,900,910,\Grid_images\F403_50X\F403_50X_grid_12.tif,3,GRID00000000003,99
4,F403,,13,50X,890,905,\Grid_images\F403_50X\F403_50X_grid_13.tif,4,GRID00000000004,99


Creating slide ID column for the grid_data table.

In [20]:
slide_ID_match = slide_data[slide_id_hub_cols].copy()

In [21]:
grid_data = grid_data.merge(slide_ID_match, how='left')
grid_data.head()

,Core,Sub_core,Grid number,Magnification,Height,Width,Grid Path,num_id,grid_id,status_code,slide_id
0,F403,,1,50X,910,895,\Grid_images\F403_50X\F403_50X_grid_1.tif,0,GRID00000000000,99,SLIDE0000000000
1,F403,,10,50X,895,905,\Grid_images\F403_50X\F403_50X_grid_10.tif,1,GRID00000000001,99,SLIDE0000000000
2,F403,,11,50X,905,904,\Grid_images\F403_50X\F403_50X_grid_11.tif,2,GRID00000000002,99,SLIDE0000000000
3,F403,,12,50X,900,910,\Grid_images\F403_50X\F403_50X_grid_12.tif,3,GRID00000000003,99,SLIDE0000000000
4,F403,,13,50X,890,905,\Grid_images\F403_50X\F403_50X_grid_13.tif,4,GRID00000000004,99,SLIDE0000000000


In [22]:
grid_data_cols = ['grid_id','slide_id','Grid number','Height','Width','status_code','Grid Path']

In [23]:
grid_data.to_csv('grid_data.csv',columns = grid_data_cols, index=False)

Ceating the annotation tables. First bind the dataframes

In [24]:
all_annotation = pd.concat([annotation_pseudo,annotation_genus, annotation_species], ignore_index=True)

In [25]:
all_annotation = all_annotation.rename(columns = {"Grid No.":"Grid number"})
all_annotation.head()

,Core,Sub_core,Grid number,Annotation type,Ostracod count,Annotation path
0,F403,,1,pseudo,8,\pseudo_annotation\pascal_voc\F403_50X\F403_50...
1,F403,,2,pseudo,5,\pseudo_annotation\pascal_voc\F403_50X\F403_50...
2,F403,,3,pseudo,2,\pseudo_annotation\pascal_voc\F403_50X\F403_50...
3,F403,,4,pseudo,6,\pseudo_annotation\pascal_voc\F403_50X\F403_50...
4,HK14DB1C,0_1,1,pseudo,2,\pseudo_annotation\pascal_voc\HK14DB1C_0_1_50X...


In [26]:
grid_ID_match = grid_data[['grid_id','Core','Sub_core','Grid number']].copy()
all_annotation = all_annotation.merge(grid_ID_match, how='left')
all_annotation.head()

,Core,Sub_core,Grid number,Annotation type,Ostracod count,Annotation path,grid_id
0,F403,,1,pseudo,8,\pseudo_annotation\pascal_voc\F403_50X\F403_50...,GRID00000000000
1,F403,,2,pseudo,5,\pseudo_annotation\pascal_voc\F403_50X\F403_50...,GRID00000000011
2,F403,,3,pseudo,2,\pseudo_annotation\pascal_voc\F403_50X\F403_50...,GRID00000000022
3,F403,,4,pseudo,6,\pseudo_annotation\pascal_voc\F403_50X\F403_50...,GRID00000000033
4,HK14DB1C,0_1,1,pseudo,2,\pseudo_annotation\pascal_voc\HK14DB1C_0_1_50X...,GRID00000000060


In [27]:
build_id_from_column('Annotation path','Annotation_id','ANNO',all_annotation)
all_annotation.head()

,Core,Sub_core,Grid number,Annotation type,Ostracod count,Annotation path,grid_id,num_id,Annotation_id
0,F403,,1,pseudo,8,\pseudo_annotation\pascal_voc\F403_50X\F403_50...,GRID00000000000,17400,ANNO00000017400
1,F403,,2,pseudo,5,\pseudo_annotation\pascal_voc\F403_50X\F403_50...,GRID00000000011,17401,ANNO00000017401
2,F403,,3,pseudo,2,\pseudo_annotation\pascal_voc\F403_50X\F403_50...,GRID00000000022,17402,ANNO00000017402
3,F403,,4,pseudo,6,\pseudo_annotation\pascal_voc\F403_50X\F403_50...,GRID00000000033,17403,ANNO00000017403
4,HK14DB1C,0_1,1,pseudo,2,\pseudo_annotation\pascal_voc\HK14DB1C_0_1_50X...,GRID00000000060,17404,ANNO00000017404


In [44]:
grid_annotations_cols = ['Annotation_id', 'grid_id', 'Annotation type', 'Annotation path']
annotation_data_cols = ['Annotation_id', 'Ostracod count']
all_annotation.to_csv('grid_annotations.csv',columns = grid_annotations_cols,index=False)
all_annotation.to_csv('annotation_data.csv',columns=annotation_data_cols,index=False)

Create the data table for genus and species. First binding the slide id and grid id into the dataset

In [30]:
specimen_data = pd.read_csv('specimen_data.csv')
specimen_data = specimen_data.transform(lambda x: x.fillna('') if x.dtype == 'object' else x.fillna(0))

In [31]:
grid_slide_id_match = grid_data[['grid_id','slide_id','Core','Sub_core','Grid number']]

In [32]:
specimen_data = specimen_data.rename(columns = {"Grid No.": "Grid number"})
specimen_data = specimen_data.merge(grid_slide_id_match, how = 'left')

In [33]:
specimen_data.head()

,Core,Sub_core,Grid number,Height,Width,Genus,Species,Specimen Path,grid_id,slide_id
0,HK14DB1C,72_73,23,117,215,aglaiocypris,aglaiocypris unidentified,\class_images\aglaiocypris unidentified\HK14DB...,GRID00000001095,SLIDE0000000018
1,HK14DB2C,0_1,12,254,144,aglaiocypris,aglaiocypris unidentified,\class_images\aglaiocypris unidentified\HK14DB...,GRID00000001503,SLIDE0000000024
2,HK14DB2C,0_1,8,260,220,aglaiocypris,aglaiocypris unidentified,\class_images\aglaiocypris unidentified\HK14DB...,GRID00000001558,SLIDE0000000024
3,HK14DB2C,104_105,10,201,277,aglaiocypris,aglaiocypris unidentified,\class_images\aglaiocypris unidentified\HK14DB...,GRID00000001561,SLIDE0000000025
4,HK14DB2C,104_105,10,169,285,aglaiocypris,aglaiocypris unidentified,\class_images\aglaiocypris unidentified\HK14DB...,GRID00000001561,SLIDE0000000025


Build ID from the specimen path for specimen ID, species for species ID and genus for genus ID

In [34]:
build_id_from_column('Specimen Path','specimen_id','SPCM',specimen_data)
build_id_from_column('Genus','genus_id','GENU',specimen_data)
build_id_from_column('Species','species_id','SPEC',specimen_data)
specimen_data.head()

,Core,Sub_core,Grid number,Height,Width,Genus,Species,Specimen Path,grid_id,slide_id,num_id,specimen_id,genus_id,species_id
0,HK14DB1C,72_73,23,117,215,aglaiocypris,aglaiocypris unidentified,\class_images\aglaiocypris unidentified\HK14DB...,GRID00000001095,SLIDE0000000018,0,SPCM00000000000,GENU00000000000,SPEC00000000000
1,HK14DB2C,0_1,12,254,144,aglaiocypris,aglaiocypris unidentified,\class_images\aglaiocypris unidentified\HK14DB...,GRID00000001503,SLIDE0000000024,0,SPCM00000000001,GENU00000000000,SPEC00000000000
2,HK14DB2C,0_1,8,260,220,aglaiocypris,aglaiocypris unidentified,\class_images\aglaiocypris unidentified\HK14DB...,GRID00000001558,SLIDE0000000024,0,SPCM00000000002,GENU00000000000,SPEC00000000000
3,HK14DB2C,104_105,10,201,277,aglaiocypris,aglaiocypris unidentified,\class_images\aglaiocypris unidentified\HK14DB...,GRID00000001561,SLIDE0000000025,0,SPCM00000000003,GENU00000000000,SPEC00000000000
4,HK14DB2C,104_105,10,169,285,aglaiocypris,aglaiocypris unidentified,\class_images\aglaiocypris unidentified\HK14DB...,GRID00000001561,SLIDE0000000025,0,SPCM00000000004,GENU00000000000,SPEC00000000000


Generate output for the database

In [35]:
specimens_cols = ['specimen_id','grid_id','slide_id','genus_id', 'species_id', 'Height', 'Width', 'Specimen Path']

In [36]:
specimen_data.to_csv('specimen_data_to_DB.csv', columns = specimens_cols, index = None)

In [37]:
all_genus = specimen_data[['Genus','genus_id']].value_counts()
all_species = specimen_data[['Species', 'species_id']].value_counts()

In [38]:
all_species

Species                      species_id     
sinocytheridea impressa      SPEC00000000175    22283
neomonoceratina delicata     SPEC00000000125     9416
propontocypris unidentified  SPEC00000000161     4888
bicornucythere bisanensis    SPEC00000000019     3035
keijella kloempritensis      SPEC00000000089     2870
                                                ...  
semicytherura cfundata       SPEC00000000168        1
parakrithe japonica          SPEC00000000146        1
pistocythereis unidentified  SPEC00000000157        1
bythoceratina cfrobusta      SPEC00000000026        1
hemicytheridea cancellata    SPEC00000000072        1
Length: 197, dtype: int64

In [39]:
all_species_data = all_species.rename_axis(['Species', 'species_id']).reset_index(name='counts')
all_genus_data = all_genus.rename_axis(['Genus', 'genus_id']).reset_index(name='counts')

In [40]:
all_species_data.to_csv('species.csv', columns = ['species_id', 'Species'], index=None)
all_genus_data.to_csv('genus.csv', columns = ['genus_id', 'Genus'], index=None)

In [41]:
all_species_data['species_id'].max()

'SPEC00000000196'

Record the maximun id for future use.

In [42]:
def decode_max_id(max_id):
    return int(max_id[5:])

In [43]:
all_ids = ['slide_id','grid_id','genus_id','species_id','specimen_id','annotation_id']
slide_id_max = slide_data['slide_id'].max()
grid_id_max = grid_data['grid_id'].max()
annotation_id_max = all_annotation['Annotation_id'].max()
specimen_id_max = specimen_data['specimen_id'].max()
species_id_max = all_species_data['species_id'].max()
genus_id_max = all_genus_data['genus_id'].max()
print(slide_id_max, grid_id_max, annotation_id_max, specimen_id_max, species_id_max, genus_id_max)

SLIDE0000000325 GRID00000019276 ANNO00000046630 SPCM00000069114 SPEC00000000196 GENU00000000078


Create the initial data by hand

Experiments

In [10]:
target_cols = slide_id_hub_cols
target_cols.remove('slide_id')

In [14]:
vc = slide_data[target_cols].value_counts()
vc

Core        Sub_core  Magnification
HK14DB1C    0_1       50X              1
subSurface  ES4       50X              1
            MS16      50X              1
            MS15      50X              1
            MS14      50X              1
                                      ..
HK14TLH2C   4_5       50X              1
            48_49     50X              1
            32_33     50X              1
            16_17     50X              1
surface     _NS5      50X              1
Length: 318, dtype: int64

In [15]:
type(vc)

pandas.core.series.Series

In [16]:
for val, cnt in vc.iteritems():
    print(val)
    print(cnt)

('HK14DB1C', '0_1', '50X')
1
('subSurface', 'ES4', '50X')
1
('subSurface', 'MS16', '50X')
1
('subSurface', 'MS15', '50X')
1
('subSurface', 'MS14', '50X')
1
('subSurface', 'MS13', '50X')
1
('subSurface', 'MS1', '50X')
1
('subSurface', 'JS2', '50X')
1
('subSurface', 'ES5', '50X')
1
('subSurface', 'ES3', '50X')
1
('subSurface', 'MS2', '50X')
1
('subSurface', 'ES2', '50X')
1
('subSurface', 'ES1', '50X')
1
('subSurface', 'DS7', '50X')
1
('subSurface', 'DS4', '50X')
1
('subSurface', 'DS3', '50X')
1
('subSurface', 'DS2', '50X')
1
('subSurface', 'DS1', '50X')
1
('subSurface', 'MS17', '50X')
1
('subSurface', 'MS3', '50X')
1
('rawSample', 'B5b', '50X')
1
('subSurface', 'NS6', '50X')
1
('subSurface', 'SS2', '50X')
1
('subSurface', 'SS1', '50X')
1
('subSurface', 'PS6', '50X')
1
('subSurface', 'PS5', '50X')
1
('subSurface', 'PS4', '50X')
1
('subSurface', 'PS3', '50X')
1
('subSurface', 'PS2', '50X')
1
('subSurface', 'NS5', '50X')
1
('subSurface', 'MS4', '50X')
1
('subSurface', 'NS4', '50X')
1
('subS

In [30]:
tp_search = ('HK14DB1C', '0_1', '50X')
slide_data.index[(slide_data['Core']==tp_search[0]) & (slide_data['Sub_core']==tp_search[1]) & (slide_data['Magnification']==tp_search[2])]

Int64Index([1], dtype='int64')